In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
import matplotlib.artist
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [ ]:
df = pd.read_csv("data.csv", index_col=0)
df.index = pd.to_datetime(df.index)

In [ ]:
df

In [ ]:
means = df.resample('MS').mean()
means

In [ ]:
df_month = pd.DataFrame(index=df.resample('MS').mean().index)
df_month["mean"] = df.resample('MS').mean().val
df_month["min"] = df.resample('MS').min().val
df_month["max"] = df.resample('MS').max().val
df_month

In [ ]:
fig = plt.figure(figsize=(10, 5))  # create a figure object\n",
ax = fig.add_subplot(1, 1, 1)  # create an axes object in the figure\n",

ax.plot(df_month.index, df_month["mean"], linewidth=4)
ax.fill_between(df_month.index, df_month["max"], df_month["min"], color="#dcdcdc")

#remove top and right axis\n",
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)

#set axis limits and ticks\n",
ax.set_ylim(0,40)
ax.set_yticks(range(0,41,5))

#format axis tick labels\n",
#ax.yaxis.set_major_formatter(ticker.StrMethodFormatter('{x:.0f}'))  
    
ax.xaxis.set_major_locator(mdates.MonthLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b'))

fig.canvas.draw()

In [ ]:
df2 = df_month.copy()
df2["month"] = df2.index.month_name().map(lambda x : x[:3])
df2["year"] = df2.index.year
df2.set_index(["year","month"], inplace=True)
df2

In [ ]:
from itertools import groupby
from matplotlib.lines import Line2D

    
def label_len(my_index,level):
    labels = my_index.get_level_values(level)
    return [(k, sum(1 for i in g)) for k,g in groupby(labels)]

def make_multiindex_axis_label(ax, index):
    ax.set_xticklabels('')
    ax.set_xlabel('')
    ypos = -.1
    scale = 1./index.size
    for level in range(index.nlevels)[::-1]:
        pos = 0
        for label, rpos in label_len(index,level):
            lxpos = (pos + .5 * rpos)*scale
            ax.text(lxpos, ypos, label, ha='center', transform=ax.transAxes)
            ax.add_line(Line2D([pos*scale, pos*scale], [ypos + .1, ypos],
                      transform=ax.transAxes, color='gray', clip_on=False)
               )
            pos += rpos
        ax.add_line(Line2D([pos*scale, pos*scale], [ypos + .1, ypos],
                      transform=ax.transAxes, color='gray', clip_on=False))
        ypos -= .1


In [ ]:
fig = plt.figure(figsize=(10, 5))  # create a figure object\n",
ax = fig.add_subplot(1, 1, 1)  # create an axes object in the figure\n",

ax.plot(df_month.index, df_month["mean"], linewidth=4, color="#2d2d2d")
ax.fill_between(df_month.index, df_month["max"], df_month["min"], color="#dcdcdc")
ax.scatter(df_month.index[-1], df_month["mean"][-1], s=10**2, color="#2d2d2d")

#remove top and right axis\n",
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)

#set axis limits and ticks\n",
ax.set_ylim(0,40)
ax.set_yticks(range(0,41,5))

ax.set_ylabel('Wait time (minutes)', fontsize = 12, horizontalalignment='right', x=-0.13, y=1.02)

#format axis tick labels\n",
#ax.yaxis.set_major_formatter(ticker.StrMethodFormatter('{x:.0f}'))   
    
ax.xaxis.set_major_locator(mdates.MonthLocator())
#ax.xaxis.set_major_formatter(mdates.DateFormatter('%b'))

ax.annotate(f'AVG', (df_month.index[0],df_month["mean"][0]), xycoords='data', verticalalignment='bottom', fontweight='bold') 
ax.annotate(f'MAX', (df_month.index[0],df_month["max"][0]), xycoords='data', verticalalignment='bottom', c='#6b6b6b') 
ax.annotate(f'MIN', (df_month.index[0],df_month["min"][0]), xycoords='data', verticalalignment='bottom', c='#6b6b6b') 
ax.annotate(f'   {df_month["mean"][-1]:.0f}', (df_month.index[-1],df_month["mean"][-1]), xycoords='data', verticalalignment='center') 

# Fist title
fig.suptitle('Passport Control Wait Time', fontsize = 16, horizontalalignment='left', x=0.063, y=1.02)
# Second title
ax.set_title('Past 13 months', fontsize = 14, horizontalalignment='left', x=-0.08, y=1.05, color='#6b6b6b')
            
make_multiindex_axis_label(ax, df2.index)
fig.canvas.draw()